## Setup model

### Initialize electrode locations

In [1]:
import numpy as np
import pandas as pd
import h5py

# elec_pos = np.column_stack((np.zeros(96),np.linspace(-1900,1900,96),np.zeros(96)))

# # save electrode position to file
# elec_file = 'sim_details/linear_electrode.csv'
# df = {'channel':np.arange(elec_pos.shape[0])}
# df.update({x+'_pos':elec_pos[:,i] for i,x in enumerate(list('xyz'))})
# df = pd.DataFrame(df)
# df.to_csv(elec_file,sep=' ',index=False)

with h5py.File('Electrode2D.h5', "r") as f:
    elec_file = 'sim_details/linear_electrode.csv'  # It isn't linear but in order to not change configs I kept the name
    elec_pos = np.hstack((f['coord'][:], np.zeros((384,1))))
    df = {'channel':np.arange(elec_pos.shape[0])}
    df.update({x+'_pos':elec_pos[:,i] for i,x in enumerate(list('xyz'))})
    df = pd.DataFrame(df)
    df.to_csv(elec_file,sep=' ',index=False)

### Copy model files

In [2]:
import os, shutil

shutil.copy('rotate_morphology.swc','sim_details/components/morphologies')
shutil.copy('neuronal_model_496930324.json','sim_details/components/biophysical_neuron_models')

'sim_details/components/biophysical_neuron_models/neuronal_model_496930324.json'

### Select translation distance and rotation angles, and convert to euler angles in bmtk

In [3]:
import numpy as np
from scipy.spatial.transform import Rotation as R

x = 0
y = 350
z = 80
theta = 0.3
h = 0.2
phi = 1.27

Rot = R.from_euler('yxy',[theta,np.arccos(h),phi])
rot_zyx = Rot.as_euler('zyx')

### Build bmtk network

In [4]:
from bmtk.builder.networks import NetworkBuilder

net = NetworkBuilder('Cell')
net.add_nodes(cell_name = 'Cell',
              positions = np.array([[x,y,z]]),
              rotation_angle_xaxis = rot_zyx[2], # rotation cell
              rotation_angle_yaxis = rot_zyx[1],
              rotation_angle_zaxis = rot_zyx[0],
#               orientation=quaternion, # this option seems not implemented in bmtk
              potental='exc',
              model_type='biophysical',
              model_template='ctdb:Biophys1.hoc',
              model_processing='aibs_allactive',
              dynamics_params='neuronal_model_496930324.json', #from allen database
              morphology= 'rotate_morphology.swc') # rotated swc file

In [5]:
net.build()
net.save_nodes(output_dir='network')

In [6]:
source = NetworkBuilder('Source')
source.add_nodes(N=1,pop_name='Source',potential='exc',model_type='virtual')

In [7]:
source.add_edges(source={'pop_name':'Source'},target=net.nodes(),
                   connection_rule=1,
                   syn_weight=0.05,
                   delay=0.0,
                   weight_function=None,
                   target_sections=['soma'],
                   distance_range=[0.0,1000.0],
                   dynamics_params='AMPA_ExcToExc.json',
                   model_template='exp2syn')

In [8]:
source.build()
source.save_nodes(output_dir='network')
source.save_edges(output_dir='network')

#### Synaptic input spike

In [9]:
import pandas as pd

spike = pd.DataFrame(data={'node_ids':[0],'timestamps':[10.0],'population':['Source']})
spike.to_csv('sim_details/synaptic_input.csv',sep=' ',index=False)

#### Build

In [10]:
from bmtk.utils.sim_setup import build_env_bionet

build_env_bionet(base_dir='sim_details',
                 network_dir='network',
                 tstop=30.0, dt=0.025,
                 v_init=-91.63188171386719, # use e_pas in dynamics_params file
                 report_vars=['v'],
                 spikes_inputs=[('Source','sim_details/synaptic_input.csv')],
                 config_file = 'config.json',
                 include_examples=True,    # Copies components files
                 compile_mechanisms=True   # Will try to compile NEURON mechanisms
                )

/home/matt/PycharmProjects/detailed-single-cell/sim_details/components/mechanisms
modfiles/CaDynamics.mod modfiles/Ca_HVA.mod modfiles/Ca_LVA.mod modfiles/Ih.mod modfiles/Im.mod modfiles/Im_v2.mod modfiles/Kd.mod modfiles/K_P.mod modfiles/K_T.mod modfiles/Kv2like.mod modfiles/Kv3_1.mod modfiles/Nap.mod modfiles/NaTa.mod modfiles/NaTs.mod modfiles/NaV.mod modfiles/SK.mod modfiles/vecevent.mod
CaDynamics.mod Ca_HVA.mod Ca_LVA.mod Ih.mod Im.mod Im_v2.mod Kd.mod K_P.mod K_T.mod Kv2like.mod Kv3_1.mod Nap.mod NaTa.mod NaTs.mod NaV.mod SK.mod vecevent.mod
mkdir -p x86_64
 -> Compiling mod_func.c
x86_64-linux-gnu-gcc -O2   -I. -I..   -I/home/matt/PycharmProjects/detailed-single-cell/venv/lib/python3.8/site-packages/neuron/.data/include  -I/nrnwheel/openmpi/include -fPIC -c mod_func.c -o mod_func.o
 -> NMODL CaDynamics.mod
MODLUNIT=/home/matt/PycharmProjects/detailed-single-cell/venv/lib/python3.8/site-packages/neuron/.data/share/nrn/lib/nrnunits.lib \
  /home/matt/PycharmProjects/detailed-sing

ls: cannot access 'modfiles/*.inc': No such file or directory
Translating CaDynamics.mod into CaDynamics.c
Translating Ca_HVA.mod into Ca_HVA.c
Translating Ca_LVA.mod into Ca_LVA.c
Thread Safe
Thread Safe
Thread Safe
Translating Ih.mod into Ih.c
Translating Im.mod into Im.c
Translating Im_v2.mod into Im_v2.c
Thread Safe
Thread Safe
Thread Safe
Translating K_P.mod into K_P.c
Translating Kd.mod into Kd.c
Thread Safe
Translating K_T.mod into K_T.c
Thread Safe
Thread Safe
Translating Kv2like.mod into Kv2like.c
Translating NaTa.mod into NaTa.c
Translating Kv3_1.mod into Kv3_1.c
Thread Safe
Thread Safe
Thread Safe
Translating NaTs.mod into NaTs.c
Translating NaV.mod into NaV.c
Translating Nap.mod into Nap.c
NEURON's CVode method ignores conservation
Notice: LINEAR is not thread safe.
Thread Safe
Thread Safe
Translating SK.mod into SK.c
Translating vecevent.mod into vecevent.c
Notice: VERBATIM blocks are not thread safe
Thread Safe


 -> Compiling x86_64/K_P.c
(cd .. ; x86_64-linux-gnu-gcc -O2   -I. -I..   -I/home/matt/PycharmProjects/detailed-single-cell/venv/lib/python3.8/site-packages/neuron/.data/include  -I/nrnwheel/openmpi/include -fPIC -c x86_64/K_P.c -o x86_64/K_P.o)
 -> Compiling x86_64/K_T.c
(cd .. ; x86_64-linux-gnu-gcc -O2   -I. -I..   -I/home/matt/PycharmProjects/detailed-single-cell/venv/lib/python3.8/site-packages/neuron/.data/include  -I/nrnwheel/openmpi/include -fPIC -c x86_64/K_T.c -o x86_64/K_T.o)
 -> Compiling x86_64/Kd.c
(cd .. ; x86_64-linux-gnu-gcc -O2   -I. -I..   -I/home/matt/PycharmProjects/detailed-single-cell/venv/lib/python3.8/site-packages/neuron/.data/include  -I/nrnwheel/openmpi/include -fPIC -c x86_64/Kd.c -o x86_64/Kd.o)
 -> Compiling x86_64/Kv2like.c
(cd .. ; x86_64-linux-gnu-gcc -O2   -I. -I..   -I/home/matt/PycharmProjects/detailed-single-cell/venv/lib/python3.8/site-packages/neuron/.data/include  -I/nrnwheel/openmpi/include -fPIC -c x86_64/Kv2like.c -o x86_64/Kv2like.o)
 -> Com

Remember to add 'ecp_report' module to the simulation_config.json file. See example in the config file in the root directory.

In [11]:
import os, shutil
shutil.copy('simulation_config.json','sim_details/simulation_config.json')

'sim_details/simulation_config.json'

### Run bmtk

In [12]:
from bmtk.simulator import bionet

conf = bionet.Config.from_json('sim_details/config.json')
conf.build_env()
net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
sim.run()

2022-03-31 09:51:02,074 [INFO] Created log file


INFO:NEURONIOUtils:Created log file


2022-03-31 09:51:02,166 [INFO] Building cells.


INFO:NEURONIOUtils:Building cells.


ValueError: argument not a density mechanism name.

#### Check morphology
Function to plot morphology

In [ ]:
from mpl_toolkits import mplot3d
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from matplotlib import cm
import matplotlib.pyplot as plt
%matplotlib notebook

def plot_morphology(network,gid=0,rotated=True,figsize=(8,6),ax=None):
    cell = network.get_cell_gid(gid)
    morph = cell.morphology
    if rotated:
        coords = cell.get_seg_coords()
    else:
        coords = morph.seg_coords
    stype = morph.seg_prop['type'].astype(int)-1
    st = list(morph.sec_type_swc.keys())[::2]
    ilab = [list(stype).index(i) for i in range(len(st))]
    clr = ['g','r','b','c']
    if ax is None:
        print(cell.morphology_file)
        fig = plt.figure(figsize=figsize)
        ax = plt.axes(projection='3d')
    else:
        fig = ax.figure
        fig.set_size_inches(figsize)
    for i in range(stype.size):
        label = st[ilab.index(i)] if i in ilab else None
        if stype[i]==0:
            ax.scatter(*coords['p05'][:,i].tolist(),c=clr[0],s=30,label=label)
        else:
            ax.plot3D(*[[coords['p0'][j,i],coords['p1'][j,i]] for j in range(3)],
                      color=clr[stype[i]],label=label)
    ax.legend(loc=1)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    return fig, ax

In [ ]:
%matplotlib notebook

_ = plot_morphology(net,0,rotated=True)

### Check Results

In [ ]:
import h5py
import numpy as np
import pandas as pd
import os

outpath = 'sim_details/output/'

#### Check membrane voltage

In [ ]:
filename = 'v_report.h5'
v_file = outpath + filename
v_f = h5py.File(v_file,'r')

V = v_f['report/Cell/data'][()]
t = np.arange(*v_f['report/Cell/mapping/time']) # array of time

import matplotlib.pyplot as plt
%matplotlib inline

plt.figure()
plt.plot(t,V)
plt.xlabel('time (ms)')
plt.ylabel('membrane voltage (mV)')
plt.show()

#### Check LFP

In [ ]:
filename = 'ecp.h5'
ecp_file = outpath + filename
ecp_f = h5py.File(ecp_file,'r')
lfp = ecp_f['ecp/data'][()]
t = np.arange(*v_f['report/Cell/mapping/time']) # array of time
fs = 1000/v_f['report/Cell/mapping/time'][2]

In [ ]:
from scipy import signal

filt_b,filt_a = signal.butter(2,100,'hp',fs=fs)
lfp_filt = signal.lfilter(filt_b,filt_a,lfp,axis=0)

In [ ]:
_,ax = plt.subplots(2,1,figsize=(8,6))
idx = (t>=0) & (t<=30)
ax[0].set_title('LFP from all electrodes')
for i in range(lfp.shape[1]):
    ax[0].plot(t[idx],lfp[idx,i])

idx = (t>=10) & (t<=20)
ax[1].set_title('Filtered LFP')
ax[1].set_xlabel('ms')
for i in range(lfp_filt.shape[1]):
    ax[1].plot(t[idx],lfp_filt[idx,i],color=ax[0].lines[i].get_color())

Save to h5 file

In [ ]:
hf = h5py.File('detailed_groundtruth.h5', 'w')
hf.create_dataset('data',data=lfp_filt)
hf.close()